In [34]:
import sqlite3
import requests
import flet as ft

# SQLite DBファイルのパス
DB_PATH = 'weather.db'

# 地域リスト取得用のエンドポイントURL
AREA_LIST_URL = "http://www.jma.go.jp/bosai/common/const/area.json"

# 天気予報取得用のテンプレートURL
WEATHER_URL_TEMPLATE = "https://www.jma.go.jp/bosai/forecast/data/forecast/{}.json"

# 天気画像のパス
WEATHER_IMAGE_PATH = "weather.png"  # このファイルは同じディレクトリに保存

# DB初期化（テーブル作成）
def init_db():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS regions (
        area_code TEXT PRIMARY KEY,
        name TEXT NOT NULL
    )''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS weather_forecasts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        area_code TEXT NOT NULL,
        forecast_date TEXT NOT NULL,
        weather_condition TEXT NOT NULL,
        max_temp REAL,
        min_temp REAL,
        icon_url TEXT,
        FOREIGN KEY (area_code) REFERENCES regions (area_code),
        UNIQUE(area_code, forecast_date)
    )''')

    conn.commit()
    conn.close()

# 地域リストをデータベースに格納
def save_area_list(area_list):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    for area_code, area_info in area_list['centers'].items():
        cursor.execute('''
        INSERT OR REPLACE INTO regions (area_code, name)
        VALUES (?, ?)''', (area_code, area_info['name']))

    conn.commit()
    conn.close()

# 天気予報をデータベースに格納
def save_weather_forecast(area_code, forecast_date, weather_condition, max_temp, min_temp, icon_url):
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    cursor.execute('''
    INSERT OR REPLACE INTO weather_forecasts (area_code, forecast_date, weather_condition, max_temp, min_temp, icon_url)
    VALUES (?, ?, ?, ?, ?, ?)''', (area_code, forecast_date, weather_condition, max_temp, min_temp, icon_url))

    conn.commit()
    conn.close()

# 地域リストを取得する関数
def fetch_area_list():
    try:
        response = requests.get(AREA_LIST_URL)
        response.raise_for_status()  # エラーがあれば例外を発生
        return response.json()  # JSON形式で返す
    except requests.exceptions.RequestException as e:
        print(f"地域リストの取得に失敗: {e}")
        return None

# 地域の天気予報を取得する関数
def fetch_weather_forecast(area_code):
    try:
        url = WEATHER_URL_TEMPLATE.format(area_code)
        response = requests.get(url)
        response.raise_for_status()  # エラーがあれば例外を発生
        return response.json()  # JSON形式で返す
    except requests.exceptions.RequestException as e:
        print(f"天気予報の取得に失敗: {e}")
        return None

# 天気予報を表示し、DBに格納する関数
def show_weather_forecast(page: ft.Page, area_code: str):
    # 天気予報を取得
    weather_data = fetch_weather_forecast(area_code)
    
    if not weather_data:
        page.add(ft.Text("天気予報を取得できませんでした。"))
        return

    # 天気予報の情報を抽出
    forecast_info = ""
    weather_icon = WEATHER_IMAGE_PATH  # 初期画像パス

    # 天気情報を抽出して表示
    for forecast in weather_data:
        area_name = forecast['area']['name']
        time_series = forecast['timeSeries'][0]
        weather_condition = time_series['areas'][0]['weather']
        forecast_date = time_series['timeDefines'][0]
        max_temp = time_series['areas'][0].get('temperature', {}).get('max', {}).get('value', None)
        min_temp = time_series['areas'][0].get('temperature', {}).get('min', {}).get('value', None)

        forecast_info += f"{area_name}の天気予報:\n"
        forecast_info += f"  - 発表時刻: {forecast_date}\n"
        forecast_info += f"  - 天気: {weather_condition}\n"
        forecast_info += f"  - 最高気温: {max_temp}°C\n"
        forecast_info += f"  - 最低気温: {min_temp}°C\n"

        # 天気に応じた画像を変更
        if "晴れ" in weather_condition:
            weather_icon = "weather_clear.png"
        elif "雨" in weather_condition:
            weather_icon = "weather_rain.png"
        elif "曇り" in weather_condition:
            weather_icon = "weather_cloudy.png"
        
        # DBに保存
        save_weather_forecast(area_code, forecast_date, weather_condition, max_temp, min_temp, weather_icon)

    # 天気予報情報をページに表示
    page.controls.clear()  # 既存のコンポーネントをクリア
    page.add(
        ft.Image(src=weather_icon, width=200),
        ft.Text(forecast_info, size=20)
    )

# メインアプリケーションの関数
def main(page: ft.Page):
    page.title = "気象庁 天気予報アプリ"
    page.vertical_alignment = ft.MainAxisAlignment.START

    # DB初期化
    init_db()

    # 地域リストを取得
    area_list = fetch_area_list()
    if not area_list:
        page.add(ft.Text("地域リストを取得できませんでした。"))
        return

    # 地域リストをデータベースに格納
    save_area_list(area_list)

    # 地域リストをビューに表示
    area_buttons = []
    for area_code, area_info in area_list['centers'].items():
        area_buttons.append(
            ft.ElevatedButton(
                text=area_info['name'],
                on_click=lambda e, code=area_code: show_weather_forecast(page, code)
            )
        )

    # 地域ボタンをページに追加
    page.add(ft.Column(area_buttons, spacing=10))

# Fletアプリケーションを起動
ft.app_async(target=main)

<coroutine object app_async at 0x1219aa340>